In [302]:
#!pip install -r requirements.txt

In [303]:
import pandas as pd
import numpy as np

In [304]:
#Exibir totas as colunas no pandas
pd.set_option('display.max_columns', None)

### Partidas

In [305]:
df_partidas_raw = pd.read_parquet('data/raw/brasileirao_2023/partidas/')

In [322]:
df_partidas_raw[['teams.home.name', 'teams.home.id']]

,teams.home.name,teams.home.id
0,Flamengo,127
1,Botafogo,120
2,Palmeiras,121
3,Corinthians,131
4,RB Bragantino,794
...,...,...
375,Cruzeiro,135
376,Internacional,119
377,Coritiba,147
378,Bahia,118


In [306]:
df_partidas = df_partidas_raw.copy()

df_partidas['fixture.date'] = df_partidas['fixture.date'].dt.date
df_partidas['league.round'] = df_partidas['league.round'].replace('Regular Season - ', '', regex=True)

df_partidas = df_partidas[['fixture.id', 'fixture.date', 'league.round',
                           'teams.home.name', 'teams.home.winner', 
                           'teams.away.name', 'teams.away.winner',
                           'score.fulltime.home', 'score.fulltime.away']]

df_partidas = df_partidas.astype({'league.round':'int',
                                  'teams.home.winner': 'bool',
                                  'teams.away.winner': 'bool'})

In [307]:
df_partidas.sort_values('fixture.id').head()

,fixture.id,fixture.date,league.round,teams.home.name,teams.home.winner,teams.away.name,teams.away.winner,score.fulltime.home,score.fulltime.away
0,1005649,2023-04-16,1,Flamengo,True,Coritiba,False,3,0
1,1005650,2023-04-15,1,Botafogo,True,Sao Paulo,False,2,1
2,1005651,2023-04-15,1,Palmeiras,True,Cuiaba,False,2,1
3,1005652,2023-04-16,1,Corinthians,True,Cruzeiro,False,2,1
4,1005653,2023-04-15,1,RB Bragantino,True,Bahia,False,2,1


### Estatisticas

In [308]:
df_partidas_stats_raw = pd.read_parquet('data/raw/brasileirao_2023/partidas_estatistica/')

In [309]:
df_partidas_stats = df_partidas_stats_raw.copy()

df_partidas_stats['value'] = df_partidas_stats['value'].replace('%', '', regex=True)
df_partidas_stats['value'] = df_partidas_stats['value'].replace('None', 0, regex=True)
df_partidas_stats['value'] = df_partidas_stats['value'].astype('float')

df_partidas_stats.loc[
    df_partidas_stats['type'].isin(['Ball Possession', 'Passes %']), 'value'
] = df_partidas_stats['value']/100

In [310]:
df_wide = pd.pivot(df_partidas_stats, index=['fixture.id', 'team.name'], columns='type', values='value').reset_index()

In [311]:
df_wide

type,fixture.id,team.name,Ball Possession,Blocked Shots,Corner Kicks,Fouls,Goalkeeper Saves,Offsides,Passes %,Passes accurate,Red Cards,Shots insidebox,Shots off Goal,Shots on Goal,Shots outsidebox,Total Shots,Total passes,Yellow Cards,expected_goals
0,1005649,Coritiba,0.41,5.0,3.0,17.0,4.0,2.0,0.85,322.0,0.0,3.0,2.0,3.0,7.0,10.0,379.0,3.0,0.36
1,1005649,Flamengo,0.59,3.0,6.0,20.0,3.0,1.0,0.89,480.0,0.0,5.0,5.0,7.0,10.0,15.0,542.0,3.0,1.37
2,1005650,Botafogo,0.32,4.0,3.0,20.0,7.0,2.0,0.78,229.0,0.0,6.0,5.0,3.0,6.0,12.0,293.0,3.0,0.84
3,1005650,Sao Paulo,0.68,2.0,7.0,13.0,1.0,1.0,0.89,574.0,0.0,11.0,6.0,8.0,5.0,16.0,646.0,2.0,1.86
4,1005651,Cuiaba,0.48,2.0,4.0,12.0,3.0,2.0,0.79,337.0,1.0,6.0,6.0,4.0,6.0,12.0,426.0,5.0,0.67
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
755,1006026,Coritiba,0.46,3.0,6.0,17.0,4.0,0.0,0.90,406.0,0.0,8.0,5.0,5.0,5.0,13.0,451.0,1.0,0.49
756,1006027,Atletico-MG,0.63,2.0,5.0,10.0,2.0,3.0,0.85,463.0,0.0,4.0,0.0,5.0,3.0,7.0,546.0,2.0,0.97
757,1006027,Bahia,0.37,2.0,3.0,16.0,4.0,2.0,0.75,238.0,0.0,11.0,7.0,6.0,4.0,15.0,318.0,2.0,1.50
758,1006028,Atletico Paranaense,0.56,8.0,9.0,15.0,3.0,2.0,0.88,444.0,0.0,10.0,6.0,3.0,7.0,17.0,507.0,3.0,0.99


Variaveis de Interesse

In [312]:
df_estatisticas = df_wide[['fixture.id', 'team.name', 'Ball Possession', 'Blocked Shots',
                           'Corner Kicks', 'Goalkeeper Saves', 'Passes %', 'Red Cards',
                           'Shots insidebox', 'Shots outsidebox', 'Shots off Goal',
                           'Shots on Goal', 'expected_goals'
                           ]]

In [313]:
df_estatisticas

type,fixture.id,team.name,Ball Possession,Blocked Shots,Corner Kicks,Goalkeeper Saves,Passes %,Red Cards,Shots insidebox,Shots outsidebox,Shots off Goal,Shots on Goal,expected_goals
0,1005649,Coritiba,0.41,5.0,3.0,4.0,0.85,0.0,3.0,7.0,2.0,3.0,0.36
1,1005649,Flamengo,0.59,3.0,6.0,3.0,0.89,0.0,5.0,10.0,5.0,7.0,1.37
2,1005650,Botafogo,0.32,4.0,3.0,7.0,0.78,0.0,6.0,6.0,5.0,3.0,0.84
3,1005650,Sao Paulo,0.68,2.0,7.0,1.0,0.89,0.0,11.0,5.0,6.0,8.0,1.86
4,1005651,Cuiaba,0.48,2.0,4.0,3.0,0.79,1.0,6.0,6.0,6.0,4.0,0.67
...,...,...,...,...,...,...,...,...,...,...,...,...,...
755,1006026,Coritiba,0.46,3.0,6.0,4.0,0.90,0.0,8.0,5.0,5.0,5.0,0.49
756,1006027,Atletico-MG,0.63,2.0,5.0,2.0,0.85,0.0,4.0,3.0,0.0,5.0,0.97
757,1006027,Bahia,0.37,2.0,3.0,4.0,0.75,0.0,11.0,4.0,7.0,6.0,1.50
758,1006028,Atletico Paranaense,0.56,8.0,9.0,3.0,0.88,0.0,10.0,7.0,6.0,3.0,0.99


In [314]:
df_partidas

,fixture.id,fixture.date,league.round,teams.home.name,teams.home.winner,teams.away.name,teams.away.winner,score.fulltime.home,score.fulltime.away
0,1005649,2023-04-16,1,Flamengo,True,Coritiba,False,3,0
1,1005650,2023-04-15,1,Botafogo,True,Sao Paulo,False,2,1
2,1005651,2023-04-15,1,Palmeiras,True,Cuiaba,False,2,1
3,1005652,2023-04-16,1,Corinthians,True,Cruzeiro,False,2,1
4,1005653,2023-04-15,1,RB Bragantino,True,Bahia,False,2,1
...,...,...,...,...,...,...,...,...,...
375,1006024,2023-12-06,38,Cruzeiro,False,Palmeiras,False,1,1
376,1006025,2023-12-06,38,Internacional,True,Botafogo,False,3,1
377,1006026,2023-12-06,38,Coritiba,False,Corinthians,True,0,2
378,1006027,2023-12-06,38,Bahia,True,Atletico-MG,False,4,1


In [315]:
df_base_final = \
    df_partidas.merge(
        df_estatisticas.add_prefix('home_'),
        how='inner',
        left_on=['fixture.id', 'teams.home.name'],
        right_on=['home_fixture.id', 'home_team.name']
    )\
    .merge(
        df_estatisticas.add_prefix('away_'),
        how='inner',
        left_on=['fixture.id', 'teams.away.name'],
        right_on=['away_fixture.id', 'away_team.name']
    )

df_base_final = df_base_final.drop(columns=['home_fixture.id', 'home_team.name', 'away_fixture.id', 'away_team.name'])

In [316]:
df_base_final

,fixture.id,fixture.date,league.round,teams.home.name,teams.home.winner,teams.away.name,teams.away.winner,score.fulltime.home,score.fulltime.away,home_Ball Possession,home_Blocked Shots,home_Corner Kicks,home_Goalkeeper Saves,home_Passes %,home_Red Cards,home_Shots insidebox,home_Shots outsidebox,home_Shots off Goal,home_Shots on Goal,home_expected_goals,away_Ball Possession,away_Blocked Shots,away_Corner Kicks,away_Goalkeeper Saves,away_Passes %,away_Red Cards,away_Shots insidebox,away_Shots outsidebox,away_Shots off Goal,away_Shots on Goal,away_expected_goals
0,1005649,2023-04-16,1,Flamengo,True,Coritiba,False,3,0,0.59,3.0,6.0,3.0,0.89,0.0,5.0,10.0,5.0,7.0,1.37,0.41,5.0,3.0,4.0,0.85,0.0,3.0,7.0,2.0,3.0,0.36
1,1005650,2023-04-15,1,Botafogo,True,Sao Paulo,False,2,1,0.32,4.0,3.0,7.0,0.78,0.0,6.0,6.0,5.0,3.0,0.84,0.68,2.0,7.0,1.0,0.89,0.0,11.0,5.0,6.0,8.0,1.86
2,1005651,2023-04-15,1,Palmeiras,True,Cuiaba,False,2,1,0.52,5.0,4.0,3.0,0.84,0.0,10.0,7.0,7.0,5.0,2.69,0.48,2.0,4.0,3.0,0.79,1.0,6.0,6.0,6.0,4.0,0.67
3,1005652,2023-04-16,1,Corinthians,True,Cruzeiro,False,2,1,0.42,1.0,7.0,1.0,0.78,0.0,6.0,6.0,4.0,7.0,1.54,0.58,3.0,4.0,5.0,0.85,0.0,4.0,4.0,3.0,2.0,0.49
4,1005653,2023-04-15,1,RB Bragantino,True,Bahia,False,2,1,0.56,2.0,4.0,2.0,0.78,0.0,6.0,7.0,4.0,7.0,2.11,0.44,1.0,7.0,5.0,0.72,0.0,5.0,4.0,5.0,3.0,0.65
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,1006024,2023-12-06,38,Cruzeiro,False,Palmeiras,False,1,1,0.56,1.0,3.0,4.0,0.83,0.0,5.0,9.0,7.0,6.0,1.41,0.44,2.0,4.0,5.0,0.80,0.0,7.0,1.0,1.0,5.0,1.30
376,1006025,2023-12-06,38,Internacional,True,Botafogo,False,3,1,0.54,1.0,1.0,4.0,0.87,0.0,7.0,0.0,3.0,3.0,1.29,0.46,2.0,5.0,0.0,0.87,0.0,5.0,4.0,2.0,5.0,0.39
377,1006026,2023-12-06,38,Coritiba,False,Corinthians,True,0,2,0.46,3.0,6.0,4.0,0.90,0.0,8.0,5.0,5.0,5.0,0.49,0.54,1.0,6.0,5.0,0.90,0.0,8.0,5.0,6.0,6.0,0.54
378,1006027,2023-12-06,38,Bahia,True,Atletico-MG,False,4,1,0.37,2.0,3.0,4.0,0.75,0.0,11.0,4.0,7.0,6.0,1.50,0.63,2.0,5.0,2.0,0.85,0.0,4.0,3.0,0.0,5.0,0.97


In [317]:
df_base_final['winner_team'] = \
    np.where( df_base_final['teams.home.winner'], df_base_final['teams.home.name'],
    np.where( df_base_final['teams.away.winner'], df_base_final['teams.away.name'],        
    'Empate'))

In [318]:
df_base_final

,fixture.id,fixture.date,league.round,teams.home.name,teams.home.winner,teams.away.name,teams.away.winner,score.fulltime.home,score.fulltime.away,home_Ball Possession,home_Blocked Shots,home_Corner Kicks,home_Goalkeeper Saves,home_Passes %,home_Red Cards,home_Shots insidebox,home_Shots outsidebox,home_Shots off Goal,home_Shots on Goal,home_expected_goals,away_Ball Possession,away_Blocked Shots,away_Corner Kicks,away_Goalkeeper Saves,away_Passes %,away_Red Cards,away_Shots insidebox,away_Shots outsidebox,away_Shots off Goal,away_Shots on Goal,away_expected_goals,winner_team
0,1005649,2023-04-16,1,Flamengo,True,Coritiba,False,3,0,0.59,3.0,6.0,3.0,0.89,0.0,5.0,10.0,5.0,7.0,1.37,0.41,5.0,3.0,4.0,0.85,0.0,3.0,7.0,2.0,3.0,0.36,Flamengo
1,1005650,2023-04-15,1,Botafogo,True,Sao Paulo,False,2,1,0.32,4.0,3.0,7.0,0.78,0.0,6.0,6.0,5.0,3.0,0.84,0.68,2.0,7.0,1.0,0.89,0.0,11.0,5.0,6.0,8.0,1.86,Botafogo
2,1005651,2023-04-15,1,Palmeiras,True,Cuiaba,False,2,1,0.52,5.0,4.0,3.0,0.84,0.0,10.0,7.0,7.0,5.0,2.69,0.48,2.0,4.0,3.0,0.79,1.0,6.0,6.0,6.0,4.0,0.67,Palmeiras
3,1005652,2023-04-16,1,Corinthians,True,Cruzeiro,False,2,1,0.42,1.0,7.0,1.0,0.78,0.0,6.0,6.0,4.0,7.0,1.54,0.58,3.0,4.0,5.0,0.85,0.0,4.0,4.0,3.0,2.0,0.49,Corinthians
4,1005653,2023-04-15,1,RB Bragantino,True,Bahia,False,2,1,0.56,2.0,4.0,2.0,0.78,0.0,6.0,7.0,4.0,7.0,2.11,0.44,1.0,7.0,5.0,0.72,0.0,5.0,4.0,5.0,3.0,0.65,RB Bragantino
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,1006024,2023-12-06,38,Cruzeiro,False,Palmeiras,False,1,1,0.56,1.0,3.0,4.0,0.83,0.0,5.0,9.0,7.0,6.0,1.41,0.44,2.0,4.0,5.0,0.80,0.0,7.0,1.0,1.0,5.0,1.30,Empate
376,1006025,2023-12-06,38,Internacional,True,Botafogo,False,3,1,0.54,1.0,1.0,4.0,0.87,0.0,7.0,0.0,3.0,3.0,1.29,0.46,2.0,5.0,0.0,0.87,0.0,5.0,4.0,2.0,5.0,0.39,Internacional
377,1006026,2023-12-06,38,Coritiba,False,Corinthians,True,0,2,0.46,3.0,6.0,4.0,0.90,0.0,8.0,5.0,5.0,5.0,0.49,0.54,1.0,6.0,5.0,0.90,0.0,8.0,5.0,6.0,6.0,0.54,Corinthians
378,1006027,2023-12-06,38,Bahia,True,Atletico-MG,False,4,1,0.37,2.0,3.0,4.0,0.75,0.0,11.0,4.0,7.0,6.0,1.50,0.63,2.0,5.0,2.0,0.85,0.0,4.0,3.0,0.0,5.0,0.97,Bahia


In [319]:
df_base_final.to_excel('base_final.xlsx')